In [84]:
from datasets import load_dataset


In [20]:
from pathlib import Path

def parse_pmc_txt(file_path):
    pmcid = file_path.stem
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()
    events, offsets = zip(*(line.strip().split("\t") for line in lines))
    return {
        "pmcid": pmcid,
        "events": list(events),
        "timestamp": [float(x) for x in offsets],
    }

# Folder containing txt files (e.g., pmc_txt/PMC1234.txt)
data_dir = Path("../am_18_llm/")
all_pmc_data = [parse_pmc_txt(p) for p in data_dir.glob("*.txt")]
df_pmc = pd.DataFrame(all_pmc_data)

In [ ]:
content_dir = '../data/text/'
def build_sample(pmcid):
    content_path = content_dir + f"{pmcid[3:]}.txt"

    with open(content_path, "r", encoding="utf-8") as f:
        content = f.read().strip()

    return {
        "pmcid": pmcid,
        "content": content
    }

# Step 3: Build dataset
contents = []
for pmcid in df_m["pmcid"]:
    contents.append(build_sample(pmcid))

df = pd.DataFrame(contents)


In [25]:
# df_labels.head()
df_labels = pd.read_csv("../am_risk_annote0310.csv")  # columns: ["pmcid", "label"]
df_label = df_labels.rename(columns={'filename': 'pmcid'})

df_merged = df_pmc.merge(df_label, on="pmcid")
df_mc = df.merge(df_merged, on="pmcid")
df_m = df_m.rename(columns={'events':'event'})

desired_order = ['pmcid', 'risk', 'event','timestamp']
df_m=df_m[desired_order]

In [80]:
from datasets import Dataset, DatasetDict
# df_merged.iloc[0]['events']
from datasets import Dataset

hf_dataset = Dataset.from_pandas(df_mc.drop(columns=["__index_level_0__"], errors="ignore"))

dataset_dict = DatasetDict({"full": hf_dataset})  # or use your own splits
from huggingface_hub import login
login(token="***")
dataset_dict.push_to_hub("juliawang2024/longcovid-risk-eventtimeseries")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/423 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/juliawang2024/longcovid-risk-eventtimeseries/commit/ca64a88ce883554d583219c9fa04c12e8c55146d', commit_message='Upload dataset', commit_description='', oid='ca64a88ce883554d583219c9fa04c12e8c55146d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/juliawang2024/longcovid-risk-eventtimeseries', endpoint='https://huggingface.co', repo_type='dataset', repo_id='juliawang2024/longcovid-risk-eventtimeseries'), pr_revision=None, pr_num=None)